# Lecture 12 - Sampling Models

In [3]:
using Pkg

In [5]:
Pkg.add("DifferentialEquations")

  Updating registry at `C:\Users\masahiro\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % [==>                                      ]  3.8 %]  9.7 %15.0 % [========>                                ]  19.8 % %5 %37.4 %Fetching: [==================>                      ]  43.6 %Fetching: [=====================>                   ]  51.9 %]  55.6 % %Fetching: [===========================>             ]  65.1 %71.3 %Fetching: [===============================>         ]  75.0 % %Fetching: [==================================>      ]  84.4 %Fetching: [========================================>]  98.0 % Resolving package versions...
 Installed DimensionalPlotRecipes ────── v1.0.0
 Installed ArrayInterface ────────────── v1.2.1
 Installed DiffEqCallbacks ───────────── v2.8.0
 Installed Inflate ───────────────────── v0.1.1
 Installed NLsolve ───────────────────── v4.1.0
 Installed D

  [19fa3120] + VertexSafeGraphs v0.1.0
  [9abbd945] + Profile 
  Building Sundials → `C:\Users\masahiro\.julia\packages\Sundials\CRi5j\deps\build.log`


In [6]:
Pkg.add("ParameterizedFunctions")

 Resolving package versions...
 Installed ParameterizedFunctions ─ v4.2.1
 Installed SymEngine ────────────── v0.7.0
 Installed FillArrays ───────────── v0.7.3
 Installed BlockBandedMatrices ──── v0.5.1
 Installed LazyArrays ───────────── v0.12.2
 Installed MatrixFactorizations ─── v0.2.0
 Installed BlockArrays ──────────── v0.10.1
 Installed BandedMatrices ───────── v0.12.3
  Updating `C:\Users\masahiro\.julia\environments\v1.2\Project.toml`
  [65888b18] + ParameterizedFunctions v4.2.1
  Updating `C:\Users\masahiro\.julia\environments\v1.2\Manifest.toml`
  [aae01518] ↑ BandedMatrices v0.10.1 ⇒ v0.12.3
  [8e7c35d0] ↑ BlockArrays v0.9.1 ⇒ v0.10.1
  [ffab5731] ↑ BlockBandedMatrices v0.4.6 ⇒ v0.5.1
  [1a297f60] ↑ FillArrays v0.6.4 ⇒ v0.7.3
  [5078a376] ↑ LazyArrays v0.10.0 ⇒ v0.12.2
  [a3b82374] ↑ MatrixFactorizations v0.1.0 ⇒ v0.2.0
  [65888b18] + ParameterizedFunctions v4.2.1
  [123dc426] + SymEngine v0.7.0
  Building SymEngine → `C:\Users\masahiro\.julia\packages\SymEngine\zSUGO\deps\b

In [7]:
using DifferentialEquations, ParameterizedFunctions, Distributions, Random, Plots; gr();

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1242
┌ Info: Precompiling ParameterizedFunctions [65888b18-ceab-5e60-b2b9-181511a3b968]
└ @ Base loading.jl:1242
┌ Info: Recompiling stale cache file C:\Users\masahiro\.julia\compiled\v1.2\Distributions\xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\masahiro\.julia\compiled\v1.2\Plots\ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1240


## Mass-Spring Model

In [8]:
MS_rhs = @ode_def begin
    dx = y
    dy = -(k/m)*x - (g/m)*y
end m k g;

In [9]:
function solve_MS(m,k,g,u0,u1)
    u0 = [u0, u1]
    tspan = (0.0, 50000.0)
    p = [m, k, g]
    prob = ODEProblem(MS_rhs, u0, tspan, p)
    sol = solve(prob)
    return sol
end;

In [10]:
solve_MS(10,0.5,0.1,4.0,0.0)
@time sol = solve_MS(10,0.5,0.1,4.0,0.0);

  0.011215 seconds (54.92 k allocations: 5.273 MiB)


In [11]:
function eqtime_MS(m,k,g,u0,u1)
    sol = solve_MS(m,k,g,u0,u1)
    threshold = 0.00001
    eqtime = findfirst([sum(y) for y in [abs.(x) for x in sol.u]] .< threshold)
    return sol.t[eqtime]
end;

In [ ]:
eqtime_MS(10,0.1,0.1,4.0,0.0)
@time eqtime_MS(10,0.1,0.1,4.0,0.0);

## SIR Model

In [ ]:
SIR_rhs = @ode_def begin
    ds = - c * s * i
    di = c * s * i - h * i
    dr = h * i
end c h;

In [ ]:
function solve_SIR(c,h)
    u0 = [0.99999 0.000001 0.0]
    tspan = (0, 5000.0)
    p = [c , h]
    prob = ODEProblem(SIR_rhs, u0, tspan, p)
    sol = solve(prob)
    return sol
end;

In [ ]:
solve_SIR(0.5,0.2)
@time sol = solve_SIR(0.5,0.1);

In [ ]:
function eqpts_SIR(c,h)
    sol = solve_SIR(c,h) 
    return sol.u[end]
end;

In [ ]:
eqpts_SIR(0.5,0.1)
@time eqpts_SIR(0.5,0.2);

## Sobol G Function

Defined as

$$ f(\vec{x}) = \prod_{i=1}^N \frac{|4x_i - 2| + a_i}{1-a_i} $$

where

$$ \vec{x} = [x_1, x_2, \cdots, x_N] , \quad x_i \sim \mathbf{U}(0,1)$$

and

$$ a_i = \frac{i - 2}{2} \text{ for all } i = 1, \cdots, N .$$

In [ ]:
function solve_SB(x; a=[0, 0.25, 99.0])
    if length(x) != length(a)
        error("Vectors must be the same dimension")
    else
        y = prod([(abs(4*x[i] - 2) + a[i])./(1+a[i]) for i = 1:length(x)])
    end
    return y
end;

In [ ]:
solve_SB([0.1 0.2 0.3])
@time sol = solve_SB([0.1 0.2 0.4]);